### Adapt the orcasound annotations file to our format

In [1]:
import pandas as pd
import numpy as np

In [2]:
annotation_file = pd.read_csv('/home/moshe/PycharmProjects/Deep_Voice/finding_willy/data/TrainDataLatest_PodCastAllRounds/annotations.tsv', sep='\t')
example_file = pd.read_csv('/home/moshe/PycharmProjects/Deep_Voice/finding_willy/EDA/combined_annotations.csv')

In [3]:
annotation_file.head()

,dataset,wav_filename,start_time_s,duration_s,location,date,pst_or_master_tape_identifier
0,podcast_round1,60012.wav,34.126,2.918,"Dabob Bay, Seattle, Washington",1960-10-28,60012
1,podcast_round1,60012.wav,36.816,2.588,"Dabob Bay, Seattle, Washington",1960-10-28,60012
2,podcast_round1,60012.wav,42.550,2.055,"Dabob Bay, Seattle, Washington",1960-10-28,60012
3,podcast_round1,60012.wav,44.606,2.410,"Dabob Bay, Seattle, Washington",1960-10-28,60012
4,podcast_round1,60012.wav,46.636,3.425,"Dabob Bay, Seattle, Washington",1960-10-28,60012


In [4]:
example_file.head()

,begin_time,end_time,filename,call_length,label
0,745.344984,826.061974,2019BG_ZOOM0002_Tr2,80.716989,0
1,189.979514,216.812778,2019BG_ZOOM0003_Tr1,26.833265,0
2,236.132729,256.526010,2019BG_ZOOM0003_Tr1,20.393281,0
3,270.479308,457.238830,2019BG_ZOOM0003_Tr1,186.759522,0
4,494.805400,639.705030,2019BG_ZOOM0003_Tr1,144.899629,0


In [5]:
annotation_file.rename(columns={'wav_filename': 'filename', 
                                'start_time_s':'begin_time', 
                                'duration_s': 'call_length'}, inplace=True)

In [6]:
annotation_file.head()

,dataset,filename,begin_time,call_length,location,date,pst_or_master_tape_identifier
0,podcast_round1,60012.wav,34.126,2.918,"Dabob Bay, Seattle, Washington",1960-10-28,60012
1,podcast_round1,60012.wav,36.816,2.588,"Dabob Bay, Seattle, Washington",1960-10-28,60012
2,podcast_round1,60012.wav,42.550,2.055,"Dabob Bay, Seattle, Washington",1960-10-28,60012
3,podcast_round1,60012.wav,44.606,2.410,"Dabob Bay, Seattle, Washington",1960-10-28,60012
4,podcast_round1,60012.wav,46.636,3.425,"Dabob Bay, Seattle, Washington",1960-10-28,60012


In [7]:
annotation_file['end_time'] = annotation_file['begin_time'] + annotation_file['call_length']

In [8]:
annotation_file['label'] = np.ones((len(annotation_file),), dtype=int)

#### reorder the columns

In [9]:
cols = annotation_file.columns.tolist()
orig_cols = example_file.columns.tolist()
remaining = list(set(cols) - set(orig_cols))
new_cols = orig_cols + remaining

In [10]:
annotation_file = annotation_file[new_cols]

#### remove .wav extension from the file names

In [11]:
name_series = [x.replace('.wav', '') for x in annotation_file['filename']]
annotation_file['filename'] = name_series

### label bg as bg, solve begin_time and call_len issues

In [17]:
annotation_file.loc[annotation_file['call_length'] == 0, 'label'] = 0

In [18]:
bg_files = list(annotation_file[annotation_file['label'] == 0]['filename'])

In [19]:
len(bg_files)

55

In [20]:
import soundfile as sf

In [21]:
wavs_path = '/home/moshe/PycharmProjects/Deep_Voice/finding_willy/data/TrainDataLatest_PodCastAllRounds/wav'

In [32]:
for bg_file in bg_files:
    annotation_file.loc[annotation_file['filename'] == bg_file, 'end_time'] = sf.info(f'{wavs_path}/{bg_file}.wav').duration
    annotation_file.loc[annotation_file['filename'] == bg_file, 'call_length'] = sf.info(f'{wavs_path}/{bg_file}.wav').duration

In [33]:
sf.info(f'{wavs_path}/{bg_files[0]}.wav').duration * 20000

74368.0

In [34]:
list(bg_files)

['9750400O',
 '9750401G',
 '9750402R',
 '9761900E',
 '9761900O',
 '9761900U',
 '9769201O',
 '9769202C',
 '9774300J',
 'OS_9_27_2017_08_03_00__0002',
 'OS_9_27_2017_08_03_00__0003',
 'OS_9_27_2017_08_09_00__0000',
 'OS_9_27_2017_08_14_00__0003',
 'OS_9_27_2017_08_35_00__0000',
 'OS_9_27_2017_10_07_00__0004',
 'OS_9_27_2017_10_28_00__0002',
 'OS_9_27_2017_10_55_00__0002',
 'OS_9_27_2017_10_55_00__0003',
 'OS_9_27_2017_11_00_00__0000',
 'OS_9_27_2017_11_00_00__0001',
 'OS_9_27_2017_11_00_00__0002',
 'OS_9_27_2017_11_00_00__0003',
 'OS_9_27_2017_11_16_00__0000',
 'OS_9_27_2017_11_16_00__0001',
 'OS_9_27_2017_11_16_00__0003',
 'OS_9_27_2017_11_16_00__0004',
 'OS_9_27_2017_11_21_00__0000',
 'OS_9_27_2017_11_21_00__0001',
 'OS_9_27_2017_11_21_00__0003',
 'OS_9_27_2017_11_21_00__0004',
 'OS_9_27_2017_11_26_00__0001',
 'OS_9_27_2017_11_26_00__0003',
 'OS_9_27_2017_11_26_00__0004',
 'OS_9_27_2017_11_32_00__0000',
 'OS_9_27_2017_11_32_00__0001',
 'OS_9_27_2017_11_37_00__0000',
 'OS_9_27_2017_11_3

In [82]:
files_3 = list(annotation_file['filename'])
srs = []
for file in files_3:
    srs.append(sf.info(f'{wavs_path}/{file}.wav').samplerate)
srs = set(srs)
print(srs)

{20000}


In [77]:
sf.info(f'{wavs_path}/{bg_files[0]}.wav').samplerate

20000

In [56]:
annotation_file['label'] == 1

0        True
1        True
2        True
3        True
4        True
        ...  
4076    False
4077    False
4078    False
4079    False
4080    False
Name: label, Length: 4081, dtype: bool

In [50]:
annotation_file

,begin_time,end_time,filename,call_length,label,location,pst_or_master_tape_identifier,date,dataset
0,34.126,37.044,60012,2.918,1,"Dabob Bay, Seattle, Washington",60012,1960-10-28,podcast_round1
1,36.816,39.404,60012,2.588,1,"Dabob Bay, Seattle, Washington",60012,1960-10-28,podcast_round1
2,42.550,44.605,60012,2.055,1,"Dabob Bay, Seattle, Washington",60012,1960-10-28,podcast_round1
3,44.606,47.016,60012,2.410,1,"Dabob Bay, Seattle, Washington",60012,1960-10-28,podcast_round1
4,46.636,50.061,60012,3.425,1,"Dabob Bay, Seattle, Washington",60012,1960-10-28,podcast_round1
...,...,...,...,...,...,...,...,...,...
4076,0.000,61.250,OS_9_27_2017_11_53_00__0002,61.250,0,orcasound_lab,11:53:00,2017-09-27,podcast_round3
4077,0.000,61.250,OS_9_27_2017_11_53_00__0003,61.250,0,orcasound_lab,11:53:00,2017-09-27,podcast_round3
4078,0.000,61.250,OS_9_27_2017_11_58_00__0000,61.250,0,orcasound_lab,11:58:00,2017-09-27,podcast_round3
4079,0.000,61.250,OS_9_27_2017_11_58_00__0002,61.250,0,orcasound_lab,11:58:00,2017-09-27,podcast_round3
